In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [59]:
# Load the data from CSV using the Pandas library
df = pd.read_csv('./crop_yield.csv')
df.head()
numerical_cols = ["Rainfall_mm","Temperature_Celsius","Days_to_Harvest"]
X=df[numerical_cols]
y=df["Yield_tons_per_hectare"]


In [60]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)
X_normalized = pd.DataFrame(X_normalized, columns=numerical_cols)
df.update(X_normalized)
df.head()

C:\Users\kingc\AppData\Local\Temp\ipykernel_4924\1743674414.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.67447716  1.36802793  0.05798759 ...  0.13504878 -0.09613481
 -1.09793036]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.update(X_normalized)


,Region,Soil_Type,Crop,Rainfall_mm,Temperature_Celsius,Fertilizer_Used,Irrigation_Used,Weather_Condition,Days_to_Harvest,Yield_tons_per_hectare
0,West,Sandy,Cotton,1.335747,0.023821,False,True,Cloudy,0.674477,6.555816
1,South,Clay,Rice,1.703634,-1.312747,True,True,Rainy,1.368028,8.527341
2,North,Loam,Barley,-1.546977,0.317020,False,False,Sunny,0.057988,1.127443
3,North,Sandy,Soybean,1.681287,-1.504137,False,True,Rainy,1.599212,6.517573
4,South,Silt,Wheat,0.694233,0.569997,True,True,Cloudy,0.212110,7.248251


In [63]:
X = df[['Region','Soil_Type','Crop','Rainfall_mm','Temperature_Celsius','Fertilizer_Used','Irrigation_Used','Weather_Condition','Days_to_Harvest']]
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = encoder.fit_transform(df[categorical_columns])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))
df_encoded = pd.concat([df, one_hot_df], axis=1)
df_encoded = df_encoded.drop(categorical_columns, axis=1)
df_encoded.head()

,Rainfall_mm,Temperature_Celsius,Fertilizer_Used,Irrigation_Used,Days_to_Harvest,Yield_tons_per_hectare,Region_East,Region_North,Region_South,Region_West,...,Soil_Type_Silt,Crop_Barley,Crop_Cotton,Crop_Maize,Crop_Rice,Crop_Soybean,Crop_Wheat,Weather_Condition_Cloudy,Weather_Condition_Rainy,Weather_Condition_Sunny
0,1.335747,0.023821,False,True,0.674477,6.555816,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.703634,-1.312747,True,True,1.368028,8.527341,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,-1.546977,0.317020,False,False,0.057988,1.127443,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.681287,-1.504137,False,True,1.599212,6.517573,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.694233,0.569997,True,True,0.212110,7.248251,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [65]:
X = df_encoded[['Rainfall_mm', 
        'Temperature_Celsius', 
        'Fertilizer_Used', 
        'Irrigation_Used', 
        'Days_to_Harvest',
        'Region_East',
        'Region_West',
        'Region_North',
        'Region_South',
        'Soil_Type_Clay',
        'Soil_Type_Sandy',
        'Soil_Type_Loam',
        'Soil_Type_Silt',
        'Soil_Type_Peaty',
        'Soil_Type_Chalky',
        'Crop_Barley',
        'Crop_Cotton',
        'Crop_Maize',
        'Crop_Rice',
        'Crop_Soybean',
        'Crop_Wheat',
        'Weather_Condition_Cloudy',
        'Weather_Condition_Rainy',
        'Weather_Condition_Sunny',
         ]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)
linreg = LinearRegression()
linreg.fit(X_train, y_train)
print(linreg.intercept_)
print(linreg.coef_)
predict = linreg.predict(X_test)
mse = mean_squared_error(y_test, predict)
mae = mean_absolute_error(y_test, predict)
r2 = r2_score(y_test, predict)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("(Accuracy)R² Score:", round(r2*100),'%')

148650965563.7427
[ 1.29882184e+00  1.43875344e-01  1.50070432e+00  1.19955292e+00
  8.29337567e-04  5.41571217e+11  5.41571217e+11  5.41571217e+11
  5.41571217e+11 -1.70501584e+11 -1.70501584e+11 -1.70501584e+11
 -1.70501584e+11 -1.70501584e+11 -1.70501584e+11 -4.95417000e+11
 -4.95417000e+11 -4.95417000e+11 -4.95417000e+11 -4.95417000e+11
 -4.95417000e+11 -2.43035980e+10 -2.43035980e+10 -2.43035980e+10]
Mean Squared Error: 0.2505295158421527
Mean Absolute Error: 0.3995217385210682
(Accuracy)R² Score: 91 %


In [67]:
reduced_X = df[['Rainfall_mm', 'Temperature_Celsius', 'Fertilizer_Used', 'Irrigation_Used']]
reduced_X_train, reduced_X_test, y_train, y_test = train_test_split(reduced_X, y, test_size=0.25, random_state=3)
reduced_linreg = LinearRegression()
reduced_linreg.fit(reduced_X_train, y_train)
print(reduced_linreg.intercept_)
print(reduced_linreg.coef_)
reduced_predict = reduced_linreg.predict(reduced_X_test)
reduced_mse = mean_squared_error(y_test, predict)
reduced_mae = mean_absolute_error(y_test, predict)
reduced_r2 = r2_score(y_test, reduced_predict)

print("Mean Squared Error:", reduced_mse)
print("Mean Absolute Error:", reduced_mae)
print("(Accuracy)R² Score:", round(reduced_r2*100),'%')

3.2998495607907627
[1.29881956 0.14398073 1.50068149 1.1994694 ]
Mean Squared Error: 0.2505295158421527
Mean Absolute Error: 0.3995217385210682
(Accuracy)R² Score: 91 %


No information was lost despite reducing the features down to 4 from the original 25 as seen by the mean squared error, mean absolute error, and R^2 score being the same on both sets of features.